In [6]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

### List the existing experiments

In [10]:
client.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/cachu/OneDrive/Documents/MCPI/Tesis_MCPI/Cursos/MLOps-Zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1748061830802, experiment_id='2', last_update_time=1748061830802, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/cachu/OneDrive/Documents/MCPI/Tesis_MCPI/Cursos/MLOps-Zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747798304478, experiment_id='1', last_update_time=1747798304478, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/cachu/OneDrive/Documents/MCPI/Tesis_MCPI/Cursos/MLOps-Zoomcamp/02-experiment-tracking/mlruns/0', creation_time=1747798304470, experiment_id='0', last_update_time=1747798304470, lifecycle_stage='active', name='Default', tags={}>]

### Create a new experiment

In [9]:
client.create_experiment(name="my-cool-experiment")

'2'

### Best models for a given experiment

In [23]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse > 6.8",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

In [24]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 7f2a6c80f9b24833964331be812ef094, rmse: 6.9125
run id: b98e1bbbf7f144ad98641346b0b17c08, rmse: 11.1673
run id: a25ec4b52ded4089b4f0536d31c6c983, rmse: 11.1673


### Register a model

In [25]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [34]:
run_id = "7f2a6c80f9b24833964331be812ef094"
model_uri = f"runs/{run_id}"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748064463353, current_stage='None', description=None, last_updated_timestamp=1748064463353, name='nyc-taxi-regressor', run_id=None, run_link=None, source='runs/7f2a6c80f9b24833964331be812ef094', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [31]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version:{version.version}")

version:3


C:\Users\cachu\AppData\Local\Temp\ipykernel_12604\3805503451.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


### Set an alias for a version in a registered model

In [32]:
model_version = "1"
client.set_registered_model_alias(model_name, "test-alias", model_version)

### Update the description for a specific version

In [33]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name = model_name,
    version = model_version,
    description = f"The model version {model_version} was transitioned to test-alias alias on {date}"
)

<ModelVersion: aliases=['test-alias'], creation_timestamp=1748060512923, current_stage='None', description='The model version 1 was transitioned to test-alias alias on 2025-05-23', last_updated_timestamp=1748064196088, name='nyc-taxi-regressor', run_id='6b5dd7c8081e40469b416aae82c356bb', run_link='', source='file:///c:/Users/cachu/OneDrive/Documents/MCPI/Tesis_MCPI/Cursos/MLOps-Zoomcamp/02-experiment-tracking/mlruns/1/6b5dd7c8081e40469b416aae82c356bb/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>